In [ ]:
%matplotlib widget

In [ ]:
from bmcs_cross_section.mkappa.mkappa import MKappa
from bmcs_cross_section.api import TShape, ReinfLayer, EC2
from bmcs_cross_section.matmod import PWLConcreteMatMod, EC2ConcreteMatMod

from bmcs_beam.beam_config.boundary_conditions import BoundaryConfig
from bmcs_beam.beam_config.boundary_conditions import BoundaryConditions
from bmcs_beam.bending.deflection_profile import DeflectionProfile
from bmcs_beam.beam_config.beam_design import BeamDesign
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import bmcs_utils.api as bu

In [ ]:
def get_Fu_and_Fs(dp, mc, bl1, n_i, rho_range=(0.0025, 0.025), slenderness_range=(3, 35)):
    rho_range = np.linspace(*rho_range, n_i)
    slenderness_range = np.linspace(*slenderness_range, n_i)

    d = mc.cross_section_shape_.B -  bl1.z
    b = mc.cross_section_shape_.B
    area_g = b * d

    rho_grid, sl_grid = np.meshgrid(rho_range, slenderness_range) 
    F_u_grid = np.zeros_like(rho_grid)
    F_s_grid = np.zeros_like(rho_grid)

    _ , ax = plt.subplots()
    ax.set_xlabel(r'$w$ [mm]')
    ax.set_ylabel(r'$F$ [KN]')

    for sl_idx in range(0,len(slenderness_range)):
        for rho_idx in range(0,len(rho_range)):      
            rho = rho_grid[rho_idx, sl_idx]
            sl = sl_grid[rho_idx, sl_idx]

            print('parameter combination', rho, sl)

            # assigning the grid area (area_g) to the reinforcement area variable
            A_j_g = rho * area_g
            bl1.A = A_j_g

            # assigning the grid length (L_g) to the beam length variable
            L_g = sl * d
            dp.beam_design.L = L_g

            # running the deflection analysis 
            F_data, w_data = dp.get_Fw()

            # plotting, post-processing & saving the data
            ax.plot(w_data, F_data/1000, label = "rho={}%-sl={} ".format(rho * 100, sl))

            w_s = dp.beam_design.L / 250
            F_u = max(F_data)
            F_s = np.interp(w_s, w_data, F_data, right=F_u*2)

            F_u_grid[rho_idx,sl_idx] = F_u
            F_s_grid[rho_idx,sl_idx] = F_s
            
    return F_u_grid, F_s_grid, rho_grid, sl_grid

In [ ]:
f_ck = 30
mc = MKappa(name='moment-curvature',n_m=100, n_kappa = 100, low_kappa=0.0, high_kappa=0.00009)
mc.matrix = 'piecewise linear'
mc.matrix_.trait_set(
    E_cc = EC2.get_E_cm(f_ck), 
    E_ct = EC2.get_E_cm(f_ck),
    eps_cr = 0.000125 * ((f_ck**0.5)/(f_ck/10)**0.3)/((30**0.5)/(30/10)**0.3),
    eps_cy = EC2.get_eps_c2(f_ck),
    eps_cu = EC2.get_eps_cu2(f_ck),
    mu = 0.00
)
mc.cross_section_shape = 'rectangle'
mc.cross_section_shape_.trait_set(B = 1000, H = 300)
csl = mc.cross_section_layout
bl1 = ReinfLayer(name='steel 1', z=100, A=231.04, count=1 ,E=203000, f_sy=560)
bl1.matmod = 'carbon'
csl.add_layer(bl1)

In [ ]:
dp = DeflectionProfile(mc=mc)
dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 0

In [ ]:
dp.app()

In [ ]:
F_u_grid, F_s_grid, rho_grid, sl_grid = get_Fu_and_Fs(dp, mc, bl1, 4, rho_range=(0.01, 0.025), slenderness_range=(3, 35))

In [ ]:
_, ax = plt.subplots(1,1)

z = F_u_grid / F_s_grid - 1./0.59 # - 1./0.51

CS = ax.contour(rho_grid, sl_grid, z)#, levels=[0])
ax.clabel(CS, inline=1, fontsize=10)
ax.set_ylabel('L/d')
ax.set_xlabel(r'$\rho$ %')
ax.set_ylim(0, 35)
ax.set_xlim(0.0, 0.025)

# Draw C30 curve
# rho_range = np.arange(0.0025, 0.025, 0.0001)
# f_ck = 30
# slenderness = get_slenderness(rho_range, 0,  1, f_ck)
# ax.plot(rho_range, slenderness, c=np.random.rand(3,))

In [ ]:
np.save('F_u_grid_carbon_study' + str(f_ck) + '.npy', F_u_grid)
np.save('F_s_grid_carbon_study' + str(f_ck) + '.npy', F_s_grid)
np.save('rho_grid_carbon_study' + str(f_ck) + '.npy', rho_grid)
np.save('sl_grid_carbon_study' + str(f_ck) + '.npy', sl_grid)

In [ ]:
f_cks = [20, 30, 35, 40, 50, 70]
F_u_grids = []
F_s_grids = []
rho_grids = []
sl_grids = []
reinforcement = 'carbon'
for f_ck in f_cks:
    f_ck = str(f_ck)
    F_u_grids.append(np.load('F_u_grid_' + reinforcement + '_EC2_eq2_tension_c_Homam' + f_ck + '.npy'))
    F_s_grids.append(np.load('F_s_grid_' + reinforcement + '_EC2_eq2_tension_c_Homam' + f_ck + '.npy'))
    rho_grids.append(np.load('rho_grid_' + reinforcement + '_EC2_eq2_tension_c_Homam' + f_ck + '.npy'))
    sl_grids.append(np.load('sl_grid_' + reinforcement + '_EC2_eq2_tension_c_Homam' + f_ck + '.npy'))

In [ ]:
_, ax = plt.subplots(1,1)

ax.set_ylabel('L/d')
ax.set_xlabel(r'$\rho$ %')
ax.set_ylim(0, 35)
ax.set_xlim(0.0, 0.025)

for f_ck, F_u_grid, F_s_grid, rho_grid, sl_grid in zip(f_cks, F_u_grids, F_s_grids, rho_grids, sl_grids):
    z = F_u_grid / F_s_grid - 1./0.59
    CS = ax.contour(rho_grid, sl_grid, z, levels=[0], colors=[np.random.rand(3,)])
    CS.collections[0].set_label('C' + str(f_ck))
#     ax.clabel(CS, inline=1, fontsize=10)
rho_range = np.arange(0.0025, 0.025, 0.0001)
# f_ck = 30
slenderness = get_slenderness(rho_range, 0,  1, f_ck)
ax.plot(rho_range, slenderness, c=np.random.rand(3,), label = 'EC2 - Steel')

ax.legend()



In [ ]:
dp.beam_design.beam.applied_loads

In [ ]:
bl1.matmod_.trait_set(E=203000, f_sy=560)

In [ ]:
# mc.app()

In [ ]:
dp.beam_design.get_Q_x()

In [ ]:
dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 0


In [ ]:
dp.beam_design.beam_conf_name 

In [ ]:
M_I, kappa_I = dp.mc.inv_M_kappa
4 * M_I[-1] / dp.beam_design.L

In [ ]:
dp.F_max

In [ ]:
mc_params = dict(
            reinforcement_type= ReinforcementType.STEEL,
            n_m=100,
            n_kappa = 100,
            kappa_slider = 0.000001,
            low_kappa = 0,
            high_kappa =  0.00002,
            eps_tu = 0.002508,
            E_cc = 34000,
            E_ct = 34000,
            eps_cr = 0.000132,
            eps_cy = 0.0015312,
            mu = 0.63,
            eps_cu = 0.003498)
mc = MKappa(**mc_params)
b = 285
h = 29
mc.cross_section_shape.B = b
mc.cross_section_shape.H = h

dp = DeflectionProfile(mc=mc)
dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
# beam.apply_load(F, 0, 0)
# bc = BoundaryConditions()
# bc.get_configured_beam()
dp.beam_design.beam_conf_name.first_load_distance = 0
dp.beam_design.L = 3000
# dp.beam_design.beam.apply_load(879898787, 0, 0)
%matplotlib widget
bu.InteractiveWindow([dp,
                      dp.beam_design,
                      dp.mc,
                      dp.mc.cs_design.cross_section_layout,
                      dp.mc.cs_design.cross_section_shape,
                      dp.mc.cross_section_layout.matrix,
                      dp.mc.cross_section_layout],
                      tight_layout=True).interact()

# M = dp.beam_design.get_M_x()
# plt.plot(mc.get_kappa_M(M+1))
# plt.show()
# mc.get_kappa_M(M) + 1

# dp.beam_design.beam.solve_for_reaction_loads(R1, R2)
# print(dp.beam_design.beam.reaction_loads)
# # print(dp.beam_design.beam.plot_bending_moment())
# dp.beam_design.beam.load
# # dp.beam_design.beam.apply_load(879898787, 0, 0)
# dp.beam_design.beam


In [ ]:
mc.matrix = 'piecewise linear'
css = mc.cross_section_shape = 'T-shape'
csl = mc.cross_section_layout
bl1 = BarLayer(name='steel 1', z=10, ds=10, count=2 )
bl2 = BarLayer(name='steel 2', z=30, ds=10, count=2 )
csl.add_layer(bl1)
csl.add_layer(bl2)


In [ ]:
mc.cross_section_layout.reinforcement = [Reinforcement(eps_sy=560/203000,\
E=203000, z=26.82, A=231.04)]